# Airline staffing problem

*Original lab session by Gérard Verfaillie.*

This problem is about the staffing of an airline, i.e. assigning pilots and crew members to scheduled flights using a Constraint Satisfaction/Optimisation Problem (CSP/COP) solver (here `facile`). Three instances of the problem are given.

As a pure CSP modelling fails for the larger instance, we will solve it using a combination of CSP and Mixed-Integer Linear Programming (MILP) modelling. You may refer to past classes with the `PuLP` library for basic usage.

<div class="alert alert-warning">
    <b>Warning</b>: You may submit in French or English. Make a choice and stick to it (don't mix languages).
</div>

## Problem description

An airline operates a given **set of flights** between **pairs of airports**. Each flight must be staffed with **two pilots** and **enough cabin crew members** depending on the **type of aircraft**.

Each staff member must start its trip **from his hometown (not before 12am, midday)** and be **back home in the evening (before 12am, midnight)**. They must **not fly more than a maximum number of flights per day**, and **more than a maximum amount of time out of home** (from first take-off time to last landing time in addition to commuting time).

Each staff member may **fly as a passenger** if they just need to be transferred to another airport/city. They may as well be **assigned to just stay home**.

We will first search for a solution to this problem, then find the optimal solution that will **maximise the money** made by the airline by **minimising the number of passenger seats assigned to staff**.

## Problem data

The following data are given:
- `Nv`: total number of **flights**;
- `Na`: total number of **airports**;
- `Ni`: total number of **cities**;
- `Va`: a table associating a **city to an airport** (they may be several airports in the same city);
- `Ov` (resp. `Dv`): this table associates an **origin** (resp. destination) airport to a flight;
- `Td` (resp. `Ta`): this table associates a **take-off time** (resp. landing time) to a flight, (in round hours);
- `Dt`: this table returns **transit times between two airports of the same city** (in hours). This time is equal to 25 (more than a day, so infinite) for airports in different cities; `Dt[i][i]` is the **time required to transfer between two aircraft at airport $i$**;
- `Np`: this table associates a **number of passenger seats** to a flight;
- `Nec`: this table associates a **number of required cabin crew members** to a flight;
- `Pr`: this table associates a **price for a ticket** in a flight;
- `Ne`: total **number of staff** in the airline;
- `Ty`: this table associates 1 to a **pilot**, 0 to a **cabin crew member**;
- `Vh`: this tables associates a **hometown to each staff member**;
- `Nvmax`: maximum **number of flights per staff member per day**;
- `Dmax`: maximum **work time** (out of home) **per staff member per day**;
- `Dda`: standard **commuting time** (both ways).

<div class="alert alert-success">
    <b>Hint</b>: Three instances are given (tiny, small, normal). Start debugging with the tiny instance; when it works, try the small instance: <b>you will be graded based on the results of the small instance</b>.
</div>
<div class="alert alert-warning">
    <b>Warning</b>: Keep the normal instance for next section. (Trying it with this modelling may stall the kernel and require a restart)
</div>

Index 0 in the tables printed below means "no flight", associated to "no airport" and "no city".

In [1]:
%run airline-tiny.py

Nv = 2
Na = 2
Ni = 2
Va = [0, 1, 2]
Ov = [0, 1, 2]
Dv = [0, 2, 1]
Td = [0, 8, 12]
Ta = [0, 9, 13]
Dt = [[0, 0, 0], [0, 2, 25], [0, 25, 2]]
Dtinf = 25
Np = [0, 80, 80]
Nec = [0, 2, 2]
Pr = [0, 100, 100]
Ne = 5
Ty = [1, 1, 0, 0, 0]
Vh = [1, 1, 1, 1, 2]
Nvmax = 2
Dmax = 7
Dda = 2


In [13]:
%run airline-small.py

In [3]:
%run airline-normal.py

## CSP modelling

### Variable definitions

First, define the variables of the problem. You may associate a variable to each flight taken by a staff member on a given day.

<div class='alert alert-info'>
<b> Answer:</b><br>
For each of the $Ne$ staff members and in a day, a list of taken flights is made. A 2-D array `flights_member` is obtained:<br>
    
- `flights_member[i]` is the list of flights taken by the staff member `i`
- `flights_member[i][j]` is the j-th flight taken by the staff member `i`
    
For example: `flights_member[0] = [0,1,2,0,0,0]`<br>
    
- The `first column is artificially built with a 0` for more convenience when indexing because Python begins indices at 0. <br>
Indeed, in the next cell, "last_flight_index" is defined. If last_flight_index = **3**, then we have access to this last flight with flight[i][**3**] instead of flight[i][**2**].
- This member takes 2 flights, $1$ and $2$ because the rest of the list is filled with $0$.<br>
- The flights are integer values between `0` and `Nv` included.
- The length of this array is `Nvmax+1`, the maximum number of flights a staff member can take in a day to which we add the first column of 0 artificially created. 
</div>

In [14]:
import facile
import numpy as np

In [15]:
# The decision variables: a 2-D array
flights_member = [facile.array(
                               [facile.variable(range(Nv+1)) for i in range(Nvmax+1)]
                                ) for j in range(Ne)]

# The first column is artificially filled with 0
# For each staff member
for i in range(Ne):
    facile.constraint(flights_member[i][0]==0)
    
# "last_flight_index" is the index of the last flight non equal to 0 for a staff member
# It is obtained by computing the sum of flights non equal to 0 for a staff member
last_flight_index = [sum((flights_member[e][i] !=0) for i in range(0, Nvmax+1)) for e in range(Ne)]

In [16]:
# # Hard coded solution, used for testing on the airline-small.py case
# solution = [[0, 1, 4,6], [0, 1, 4, 6], [0, 2, 5, 0], [0,2, 5, 0], [0,3, 7, 5], [0, 3, 7, 5], [0, 1, 4,6], [0, 1, 4, 6], [0, 2, 5, 0], [0,2, 5, 0], [0,3, 7, 5], [0, 3, 7, 5] ]

# # We force "flights_member" to be equal to "solution" so that we can detect whether 
# # the constraints overconstrain the problem or not

# for i in range(Ne):
#     for j in range(Nvmax+1):
#         facile.constraint(flights_member[i][j]==solution[i][j])

### Constraints definition (50%)

<div class="alert alert-warning">
    <b>Warning</b>: Please write and explain (use Python comments) the code for each constraint in the corresponding <b>code</b> cells.<br/>Use Python comments for explanations. <b>You will be graded accordingly.</b><br/>You may add as many cells as necessary but do not move existing cells.
</div>

- Flights assigned to `0` (no flight) must be at the end of the schedule.  
  `[1, 0, 2]` could be equivalent to `[1, 2, 0]` but only the second one is valid.

<div class='alert alert-info'>
<b> Answer:</b>
    
The constraint is: `If a flight is 0, then all the following values are equal to 0`.<br>
<br>
    <b>flights_member[i][j] == 0) => (flights_member[i][j+1:end] = 0)</b><br>
<br>
This can be translated as: <br>
`A flight is not 0 otherwise the next flight_value is 0.`<br>
<br>
We do this :
    
- `for each member i`
- `for each flight j` so that, if there is a $0$, it is propagated until the end of the    array<br>

<b> For each member i:<br>
    &emsp;For each flight j:<br>
    &emsp; &emsp; flights_member[i][j] != 0<br>
    &emsp; &emsp; otherwise<br>
    &emsp; &emsp; &emsp; flights_member[i][j+1]=0</b>

The *otherwise* is written with a `| operator`.<br><br>
</div> 

In [17]:
# For each staff member
for i in range(Ne):
    # For each flight of this staff member
    for j in range(1,Nvmax): # The range reaches Nvmax because we use the index 'j+1' 
                             # and the first column is artificially made out of 0  
        
        ### Variables
        # flight[i][j] / flight[i][j+1]
        flight_i_j   = facile.array(flights_member[i])[j]
        flight_i_jp1 = facile.array(flights_member[i])[j+1]
        
        ### Constraint
        # The flight[i][j] is not 0 otherwise the next flight flight[i][j+1] is 0
        c1 = (flight_i_j!=0) | (flight_i_jp1==0)
        facile.constraint(c1)

- If a staff member flies on that day, then (s)he leaves home and returns home.

<div class='alert alert-info'>
<b> Answer:</b><br>
    
- If a staff member `does not fly on that day`, his/her first flight is equal to `0` and everything is fine.
- If a staff member flies on that day, his her/first flight is not equal to $0$ and:
    - For the `first flight`, the `city of the origin airport of is the hometown`
    - For the `last flight` different from 0, the `city of the destination airport is the hometown`

Thus, the constraint can be expressed as:<br>
<br>
<b>For each staff member:<br>
    &emsp;first_flight = 0 <br>
    &emsp;otherwise<br>
    &emsp; &emsp;city[destination_airport[first_flight]] != hometown<br>
    &emsp; &emsp;and city(destination_airport(last_flight)) == hometown</b>
<br> 
    
The *otherwise* is written with a `| operator`.
</div>

In [18]:
# For each staff member
for i in range(Ne):
    
    ### Variables
    # First flight of a staff member / Last flight of a staff member 
    first_flight = facile.array(flights_member[i])[1]
    last_flight  = facile.array(flights_member[i])[last_flight_index[i]]
    
    # Origin airport of the first flight / Destination airport of the last flight
    origin_airport_first_flight = facile.array(Ov)[first_flight]
    dest_airport_first_flight   = facile.array(Dv)[last_flight]
    
    # Origin city of the first flight / Destination city of the last flight
    origin_city_first_flight = facile.array(Va)[origin_airport_first_flight]   
    dest_city_last_flight    = facile.array(Va)[dest_airport_first_flight]
    
    # Hometown of the staff member "i"
    hometown = Vh[i]
    
    ### Constraints 
    # The first flight is equal to 0 (the staff member does not fly)
    c2_1 = first_flight==0 

    # The origin city of the first flight is the hometown of the staff member
    c2_2 = origin_city_first_flight == hometown
    
    # The destination city of the last flight is the hometown of the staff member
    c2_3 = dest_city_last_flight == hometown
    
    facile.constraint(c2_1|c2_2)
    facile.constraint(c2_1|c2_3) 

- A staff member may chain two flights iff :
    - first flight's destination airport and second flight's origin airport cities match;
    - there is enough transfer time between flights.

<div class='alert alert-info'>
<b> Answer:</b><br>
    
`First constraint: First flight's destination airport and second flight's origin airport cities match`<br>
    
- For each staff member $i$:
    - For each flight $j$:
        - The `city of the origin airport of flight[j+1]` is the same as the `city of the destination airport of flight[j]`
        - However, `flight[j+1] can also be equal to 0`. This means that $flight[j]$ was the last flight. <br>
        We have to treat this case because the city of the origin airport of a flight of value $0$ is $0$.<br>
        If it is the case, we can't have the previous point. <br>
        Therefore, we say that the `city of the destination airport of $flight[j+1]$ can be equal to $0$ or must respect the previous point.`<br>

Then, we have: <br>

<b>For each staff member:<br>
    &emsp;For each flight $j$: <br> 
        &emsp; &emsp;city[origin_airport[ flight[ j+1 ] ] ] = (0 or city[ destination_airport[ flight[j] ])</b><br>
<br>

`Second constraint: There must be enough time between two consecutive flights
`<br>

`The difference between the departure time of flight[j+1] and the arrival time of flight[j] is inferior to the time required to transfer between two aircrafts at the same airport`<br>

Thus we have:<br>
<b> For each staff member:<br>
    &emsp;For each flight $j$:<br>
    &emsp; &emsp;Dt[flight[ j ] ] [flight[ j+1 ] ] < time_arrival[flight[ j ] ] - time_departure[flight[ j+1 ] ]</b>

There is no issue if flight[j+1] equals $0$ because its time of departure or its time of arrival is $0$. Thus, the constraint is always true.

We `transform the 2-D array Dt into a 1-D array` using $np.ravel$ so that we can index it with $facile.array()$<br>

In [19]:
for i in range(Ne):
    for j in range(1,Nvmax):
        
        ### Variables
        # Flight[i][j] and the next flight
        flight_i_j = facile.array(flights_member[i])[j]
        flight_i_jp1 = facile.array(flights_member[i])[j+1]
        
        # Destination airport of flight[i][j] / Origin airport of flight[i][j+1]                           
        dest_airport_flight_i_j = facile.array(Dv)[flight_i_j]
        origin_airport_flight_i_jp1 = facile.array(Ov)[flight_i_jp1]                          

        # Destination city of flight[i][j] / Origin city of flight[i][j+1]
        dest_city_flight_i_j    = facile.array(Va)[dest_airport_flight_i_j]
        origin_city_flight_i_jp1 = facile.array(Va)[origin_airport_flight_i_jp1]

        # Reshape Dt from a 2D array into a 1D array
        Dt_flatten = np.ravel(Dt)

        # Transit time between the airport of flight[i][j] and the airport of flight[i][j+1]
        # Accessible with Dt[k][l] => Dt[k*Na+l]
        transit_time = facile.array(Dt_flatten)[dest_airport_flight_i_j * (Na+1) + origin_airport_flight_i_jp1]
        
        # Arrival time of flight[i][j] / Departure time of flight[i][j+1]
        Ta_flight_i_j = facile.array(Ta)[flight_i_j]                       
        Td_flight_i_jp1 = facile.array(Td)[flight_i_jp1]    
        
        ### Constraints
        c3_0 = origin_city_flight_i_jp1 == 0     
        c3_1 = origin_city_flight_i_jp1 == dest_city_flight_i_j
        c3_2 =  transit_time <= Td_flight_i_jp1 - Ta_flight_i_j
        
        facile.constraint(c3_0|c3_1)
        facile.constraint(c3_0|c3_2)

- A staff member may not exceed his/her total time out of home;

<div class="alert alert-info">
    <b> Answer:</b>
    
The `difference between the arrival time of the last flight and the departure time of the first flight`, to which the `commuting time` is added, is inferior or equal to the `maximum time out of home`.

<b>(Dda + Ta_last_flight - Td_first_flight) <= Dmax</b>
    
</div>

In [20]:
# For each staff member i
for i in range(Ne):
    
    ### Variables
    # First flight of a staff member / Last flight of a staff member
    first_flight = facile.array(flights_member[i])[1]
    last_flight = facile.array(flights_member[i])[last_flight_index[i]]
    
    # Departure time of the first flight / Arrival time of the last flight
    Ta_last_flight = facile.array(Ta)[last_flight]
    Td_first_flight = facile.array(Td)[first_flight]
    
    ### Constraint
    # A staff member may not exceed his/her total time out of home
    c4 = (Dda + Ta_last_flight - Td_first_flight) <= Dmax
    facile.constraint(c4)

- Each flights requires at least two pilots and enough cabin crew members.

<div class="alert alert-info">
    <b> Answer: </b>

- For each flight, we `count the number of pilots and cabin crew members` in it:
    - For a given flight, we look into the schedule of each flight member whether this flight is in it or not:<br>
    <b> "sum([x==flight for x in flights_member[i]]) for i in range(Ne)" </b>
    - We determine if this staff member is a pilot or a cabin crew member<br>
    <b> "if is_pilot(i)"</b>
    - Then we sum the number of times it appeared in total
    
<b> pilots_per_flight>=2</b><br>
<b> crew_members_per_flight>=Nec[flight]</b>
</div> 

In [21]:
# Function which returns whether the staff member is a pilot or not
def is_pilot(i):
    return (Ty[i]==1)

### Variables
# For each "flight", we count the number of pilots and the number of cabin crew members in it
for flight in range(1,Nv+1):
    pilots_per_flight = sum([sum([x==flight for x in flights_member[i]]) for i in range(Ne) if is_pilot(i)])
    crew_members_per_flight = sum([sum([x==flight for x in flights_member[i]]) for i in range(Ne) if not is_pilot(i)])
        
    ### Constraints
    # Each flight requires at least two pilots
    facile.constraint(pilots_per_flight>=2)
    
    # Each flight requies at least "Nec[flight]" cabin crew members
    facile.constraint(crew_members_per_flight>=Nec[flight])

### Problem optimisation (5%)

### Answer the problem (10%)

<div class='alert alert-warning'>
To maximise your grade, please post-process and pretty-print the results of the optimisation and check they are consistent with the constraints. <b>If you find inconsistencies you cannot solve, please point them.</b> (You will fail this question if you pretend everything is fine or if you try to hide your issues)
</div>

For each staff member, list the flights (s)he will be flying with cities and take-off/landing times.

Valid example:
```
Staff number 0: 1 [1, 2] 2 [2, 1] scheduled at [8, 9] [12, 13]
Staff number 1: ...
```
    
For each flight, list the pilots and cabin crew members flying (together with the required staffing of the aircraft).

Valid example:
```
Flight number 1: pilots {0, 1} cabin crew {2, 3} (≥ 2) ;
Flight number 2: ...
```


In [22]:
# Function which returns whether the staff member is a pilot or not
def is_pilot(i):
    return (Ty[i]==1)

# Function which prints the schedule for each crew cabin member
def print_flights_member(flights_member):
    
    # Solve the constraints problem
    flights_member = np.ravel(flights_member)
    result = facile.solve(flights_member)

    # Reshape the 1D array "result" into a 2D array
    res_array = np.array(result.solution)
    res_array = res_array.reshape(int(len(res_array)/(Nvmax+1)), Nvmax+1)

    # For each staff member, determine his/her flights, cities of the flights and time schedule 
    for m in range(Ne):
        flights_and_cities = ""
        schedule = ""
        for i in range(Nvmax+1):
            flight = res_array[m][i] 
            if flight!=0:
                flights_and_cities = flights_and_cities + "{} {} ".format(flight, [Ov[flight], Dv[flight]])    
                schedule = schedule + "{} ".format([Td[flight], Ta[flight]])

        print("Staff number {}: {}scheduled at {}\n".format(m, flights_and_cities, schedule))
    print("--------------------------------------------------\n")

    
# Function which prints the staff members for each flight 
# and the minimum number of crew members that should be on the flight
def print_pilots_crew(flights_member):
    
    # Solve the constraints problem
    flights_member = np.ravel(flights_member)
    result = facile.solve(flights_member)

    # Reshape the 1D array "result" into a 2D array
    res_array = np.array(result.solution)
    res_array = res_array.reshape(int(len(res_array)/(Nvmax+1)), Nvmax+1)
    
    # For each flight "i"
    for i in range(1,Nv+1):
        pilots_in_flight = [] # Array which will contain the pilots for a given flight
        crew_in_flight   = [] # Array which will contai nthe crew members for a given flight  
        
        # For each staff member "m"
        for m in range(Ne):
            # For each flight of this staff member
            for flight in range(Nvmax+1):
                # If the staff member "m" is in the flight "flight"
                if res_array[m][flight] == i:
                    if is_pilot(m): # If the staff member is a pilot
                        pilots_in_flight.append(m)
                    if not is_pilot(m): # If the staff member is a crew member
                        crew_in_flight.append(m)
                        
        print("Flight number {}: pilots {} cabin crew {} (≥{})\n".format(i, pilots_in_flight, crew_in_flight, Nec[i]))
        
        # Resets the two arrays for the next flight
        pilots_in_flight = []
        crew_in_flight = []
    
# Print the schedule for each crew member
# Print the staff members in each flight and the minimum number of crew members that should be on the flight
print_flights_member(flights_member)
print_pilots_crew(flights_member)


Staff number 0: 1 [1, 3] 4 [3, 5] 6 [5, 1] scheduled at [7, 8] [12, 13] [18, 20] 

Staff number 1: 1 [1, 3] 4 [3, 5] 6 [5, 1] scheduled at [7, 8] [12, 13] [18, 20] 

Staff number 2: 2 [2, 4] 5 [4, 1] scheduled at [8, 9] [23, 24] 

Staff number 3: 2 [2, 4] 5 [4, 1] scheduled at [8, 9] [23, 24] 

Staff number 4: 3 [2, 5] 7 [5, 4] 5 [4, 1] scheduled at [9, 11] [18, 19] [23, 24] 

Staff number 5: 3 [2, 5] 7 [5, 4] 5 [4, 1] scheduled at [9, 11] [18, 19] [23, 24] 

Staff number 6: 1 [1, 3] 4 [3, 5] 6 [5, 1] scheduled at [7, 8] [12, 13] [18, 20] 

Staff number 7: 1 [1, 3] 4 [3, 5] 6 [5, 1] scheduled at [7, 8] [12, 13] [18, 20] 

Staff number 8: 2 [2, 4] 5 [4, 1] scheduled at [8, 9] [23, 24] 

Staff number 9: 2 [2, 4] 5 [4, 1] scheduled at [8, 9] [23, 24] 

Staff number 10: 3 [2, 5] 6 [5, 1] scheduled at [9, 11] [18, 20] 

Staff number 11: 3 [2, 5] 7 [5, 4] 5 [4, 1] scheduled at [9, 11] [18, 19] [23, 24] 

--------------------------------------------------

Flight number 1: pilots [0, 1] cabin

## A different modelling for CSP and MILP

Since bigger instances will require too much computing time, let's try something smarter:
1. First use a CSP solver to compute all possible paths between cities;
2. Then compare performances between a CSP and a MILP solver as we no longer assign staff members but groups of staff members to each path.

<div class="alert alert-success">
    <b>Hint</b>: Three instances are given (tiny, small, normal). Start debugging with the small instance; when it works, try the normal instance: you will be graded based on the results of the <b>normal instance</b>.
</div>

### First questions (10%)

<div class="alert alert-warning">
    <b>Warning</b>: Do not expect any external assistance for these questions. You are on your own.
</div>

1. Why would it help to solve the problem for groups of staff member rather than individuals. Which mathematical principle enters into account?
2. Try to explain why CSP better suits (compared to MILP) the path computing part of the problem.

<div class = 'alert alert-info'>
<b> Answer: </b><br>

It would help to solve the problem for groups of staff member rather than individuals. This would be using `dimensionality reduction`.<br>
<br>
If the problem involves a huge number of features, the training will be extremely slow and it can be harder to find a good solution.<br>
Moreover, it is useful for data visualization and, in some cases, it can filter out sime noise and necessary details, thus result in higher performance.    
</div>


<div class = 'alert alert-info'>
<b> Answer: </b><br>
    CSP better suits compared to MILP the path computing part of the problem because:<br>

- There are `constraints which are not linear` because they contain and "or" with the | operator, or an "and" with the & operator
- The `constraints do not contain a lot of variables`. This can make CSP more efficient than MILP.
- The `domains are finite`.
- We can do a `more natural statement` of the problem with CSP.
- We can do `constraint propagation and dimension reduction`. 
</div>

### Compute all possible paths (10%)

In [23]:
import facile
import numpy as np

In [27]:
%run airline-small.py

In [25]:
%run airline-normal.py

In [28]:
# The decision variables: a 1D array containing all the possible paths between cities 
path = [facile.variable(range(Nv+1)) for _ in range(Nvmax+1)]

# The first column is artificially filled with 0
facile.constraint(path[0]==0)
    
# "last_flight_index" is the index of the last flight non equal to 0 for a staff member
# It is obtained by computing the sum of flights non equal to 0 for a staff member
last_flight_index = [sum((path[i] !=0) for i in range(0, Nvmax+1))]

<div class="alert alert-info">
    
**Constraint 1:**
- Flights assigned to 0 (no flight) must be at the end of the schedule.  
  [1, 0, 2] could be equivalent to [1, 2, 0] but only the second one is valid.

</div>

In [29]:
# For each flight in "path"
for j in range(1,Nvmax): # The range reaches Nvmax because we use the index 'j+1' 
                         # and the first column is artificially made out of 0  
    ### Variables
    # path[j] / path[j+1]
    flight_j   = facile.array(path)[j]
    flight_jp1 = facile.array(path)[j+1]
        
    ### Constraint
    # The flight[i][j] is not 0 otherwise the next flight flight[i][j+1] is 0
    c1 = (flight_j!=0) | (flight_jp1==0)
    facile.constraint(c1)

<div class="alert alert-info">

<b> Constraint 2: </b>
- A staff member may chain two flights if:
    - first flight's destination airport and second flight's origin airport cities match;
    - there is enough transfer time between flights.
    
</div>

In [30]:
# For each flight in "path"
for j in range(1,Nvmax):

    ### Variables
    # A given flight (path[j]) and the next flight (path[j+1])
    flight_j   = facile.array(path)[j]
    flight_jp1 = facile.array(path)[j+1]

    # Destination airport of path[j] / Origin airport of path[j+1]                          
    dest_airport_flight_j     = facile.array(Dv)[flight_j]
    origin_airport_flight_jp1 = facile.array(Ov)[flight_jp1]                          

    # Destination city of path[j] / Origin city of path[j+1]
    dest_city_flight_j     = facile.array(Va)[dest_airport_flight_j]
    origin_city_flight_jp1 = facile.array(Va)[origin_airport_flight_jp1]

    # Reshape Dt from a 2D array into a 1D array
    Dt_flatten = np.ravel(Dt)

    # Transit time between the airport of path[j] and the airport of path[j+1]
    # Accessible with Dt[k][l] => Dt[k*Na+l]
    transit_time = facile.array(Dt_flatten)[dest_airport_flight_j * (Na+1) + origin_airport_flight_jp1]

    # Arrival time of path[j] / Departure time of path[j+1]
    Ta_flight_j = facile.array(Ta)[flight_j]                       
    Td_flight_jp1 = facile.array(Td)[flight_jp1]    

    ### Constraints
    c3_0 = origin_city_flight_jp1 == 0     
    c3_1 = origin_city_flight_jp1 == dest_city_flight_j
    c3_2 =  transit_time <= Td_flight_jp1 - Ta_flight_j

    facile.constraint(c3_0|c3_1)
    facile.constraint(c3_0|c3_2)

<div class="alert alert-info">
    
**Constraint 3:**

- A staff member may not exceed his/her total time out of home;
    
</div>

In [31]:
### Variables
# First flight in "path" / Last flight non equal to 0 in "path"
first_flight = facile.array(path)[1]
last_flight  = facile.array(path)[last_flight_index[0]]

# Departure time of the first flight / Arrival time of the last flight
Ta_last_flight  = facile.array(Ta)[last_flight]
Td_first_flight = facile.array(Td)[first_flight]

### Constraint
# A staff member may not exceed his/her total time out of home
c4 = (Dda + Ta_last_flight - Td_first_flight) <= Dmax
facile.constraint(c4)

<div class="alert alert-info">
   
**Constraint 4**:

- If there is a flight, the last flight returns to the city of the first flight 
    
This constraint was not in the previous part but is just:<br>
<b> dest_city_last_flight == origin_city_first_flight</b>
</div>

In [32]:
### Variables
# First flight of the staff member / Last flight of the staff member
first_flight = facile.array(path)[1]
last_flight  = facile.array(path)[last_flight_index[0]]

# Origin airport of the first flight / Destination airport of the last flight
origin_airport_first_flight = facile.array(Ov)[first_flight]
dest_airport_last_flight    = facile.array(Dv)[last_flight]

# Origin city of the first flight / Destination city of the last flight 
origin_city_first_flight = facile.array(Va)[origin_airport_first_flight]
dest_city_last_flight    = facile.array(Va)[dest_airport_last_flight]

### Constraints
# The destination city of the last flight is equal to the origin city of the first flight
c4_2 = dest_city_last_flight == origin_city_first_flight
facile.constraint(c4_2) 

<div class="alert alert-info">
    
**Answer**:
    
The possible paths are generated in the variable **possible_paths**
</div>

In [33]:
# Generate the possible paths
res = facile.solve_all(path)
possible_paths = []

for x in res:
    if x.solution!= None:
        possible_paths.append(x.solution)
        
print("Possible schedules for a staff member: {}".format(possible_paths))

Possible schedules for a staff member: [[0, 0, 0, 0], [0, 1, 4, 6], [0, 2, 5, 0], [0, 3, 6, 0], [0, 3, 7, 5]]


### CSP and MILP resolution (15%)

- Assign a group of staff members to each path;
- Compare the computing time with CSP and MILP resolutions (use `PuLP` library for MILP);
- Post-process and pretty-print the results;
- Comment

<div class="alert alert-info">
    
<b> Answer:</b>   

The possible paths are contained in the variable **possible_paths**

The decision variables is flights_member, a 1-D array because, as for the variable **Dt**, a 2-D array will not work when using facile.array
- **i**: staff member
- **flights_member[i]** : number of the schedule taken by the staff member **i**<br>
It takes value in [0, len(possible_paths)]

</div>

In [34]:
# Contains the possible paths in a 1-D array. 
# As for the variable Dt, a 2-D array will not work when using facile.array
paths_var = facile.array(np.reshape(possible_paths, -1))

# Decision variable in a 1-D array
flights_member = facile.array([facile.variable(range(len(possible_paths))) for i in range(Ne)])

<div class="alert alert-info">
    
**Constraint:**

If a staff member does not fly, the first flight is 0. <br>
**first_flight==0**<br> 

Otherwise, he/she must have a schedule which first flight departs from his/her hometown.<br>
**origin_city_first_flight == hometown**
</div>

In [35]:
# For each staff member
for i in range(Ne):
    
    ### Variables
    # Schedule number in flights_member[i]
    schedule_number = flights_member[i]
    
    # First flight of the schedule / Last flight of the schedule 
    # With the 1-D array paths_var, paths_var[k][l] = paths_var[k*Nvmax+l]
    # "k" is the schedule number
    # for the first flight, "l=1"     ;    for the last flight, "l=Nvmax"
    first_flight = facile.array(paths_var)[schedule_number*Nvmax+1]
    
    # Origin airport of the first flight 
    origin_airport_first_flight = facile.array(Ov)[first_flight]
    
    # Origin city of the first flight 
    origin_city_first_flight = facile.array(Va)[origin_airport_first_flight]   
    
    # Hometown of the staff member "i"
    hometown = Vh[i]
    
    ### Constraints 
    # The first flight is equal to 0 (the staff member does not fly)
    c2_1 = first_flight==0 

    # The origin city of the first flight is the hometown of the staff member
    c2_2 = origin_city_first_flight == hometown
    
    facile.constraint(c2_1|c2_2)

<div class="alert alert-info">
    
**Constraint:**
- Each flights requires at least two pilots and enough cabin crew members.

</div>

<div class = "alert alert-warning">
    
This constraint is not working.<br>

I wanted to use the same code as the **part 1**, that is to say count the number of time a given flight appears in a schedule but the synthax is not correct:
<b> facile.array(schedules)[facile.array(flights_member)[i]]] </b>

This is supposed to give access to the **schedule** which number is in **flights_number[i]**, but the error is "The arguments must be variables or expression"
</div>

In [37]:
# Function which returns whether the staff member is a pilot or not
def is_pilot(i):
    return (Ty[i]==1)


# "schedules" is a 2-D matrix whose shape is (number_of_possibles_schedules, maximum_number_of_flights)
# schedules[k] gives the possible schedule of flights number k, for example [0,1,4,6]
# We obtain [[schedule_0] [schedule_1] [schedule_2] ...]
schedules = []
for i in range(len(possible_paths)):
    schedule_i = []
    for j in range(Nvmax+1):
        schedule_i.append(facile.array(paths_var)[flights_member[i]*Nvmax+j])
    # schedules is a 1-D array here
    schedules = np.concatenate((np.array(schedules), np.array(schedule_i)), axis=0)

number_of_possible_schedules = int(len(schedules)/(Nvmax+1))
maximum_number_of_flights = Nvmax+1
# Reshape the 1-D array into a 2-D array
schedules = schedules.reshape(number_of_possible_schedules, maximum_number_of_flights)



### Variables
# For each "flight", we count the number of pilots and the number of cabin crew members in it
for flight in range(1,Nv+1):
    pilots_per_flight = sum([sum([x==flight for x in facile.array(schedules)[facile.array(flights_member)[i]]]) for i in range(Ne) if is_pilot(i)])
    crew_members_per_flight = sum([sum([x==flight for x in facile.array(schedules)[facile.array(flights_member)[i]]]) for i in range(Ne) if not is_pilot(i)])
        
    ### Constraints
    # Each flight requires at least two pilots
    facile.constraint(pilots_per_flight>=2)
    
    # Each flight requies at least "Nec[flight]" cabin crew members
    facile.constraint(crew_members_per_flight>=Nec[flight])

TypeError: The arguments must be variables or expressions

<div class="alert alert-info">
    
**Pretty print the results**
</div>

In [283]:
# Function which returns whether the staff member is a pilot or not
def is_pilot(i):
    return (Ty[i]==1)

# Function which prints the schedule for each crew cabin member
def print_flights_member(flights_member):
    
    # Solve the constraints problem
    flights_member = np.ravel(flights_member)
    result = facile.solve(flights_member)

    # Reshape the 1D array "result" into a 2D array
    res_array = np.array(result.solution)
    res_array = res_array.reshape(int(len(res_array)/(Nvmax+1)), Nvmax+1)

    # For each staff member, determine his/her flights, cities of the flights and time schedule 
    for m in range(Ne):
        flights_and_cities = ""
        schedule = ""
        for i in range(Nvmax+1):
            flight = res_array[m][i] 
            if flight!=0:
                flights_and_cities = flights_and_cities + "{} {} ".format(flight, [Ov[flight], Dv[flight]])    
                schedule = schedule + "{} ".format([Td[flight], Ta[flight]])

        print("Staff number {}: {}scheduled at {}\n".format(m, flights_and_cities, schedule))
    print("--------------------------------------------------\n")

    
# Function which prints the staff members for each flight 
# and the minimum number of crew members that should be on the flight
def print_pilots_crew(flights_member):
    
    # Solve the constraints problem
    flights_member = np.ravel(flights_member)
    result = facile.solve(flights_member)

    # Reshape the 1D array "result" into a 2D array
    res_array = np.array(result.solution)
    res_array = res_array.reshape(int(len(res_array)/(Nvmax+1)), Nvmax+1)
    
    # For each flight "i"
    for i in range(1,Nv+1):
        pilots_in_flight = [] # Array which will contain the pilots for a given flight
        crew_in_flight   = [] # Array which will contai nthe crew members for a given flight  
        
        # For each staff member "m"
        for m in range(Ne):
            # For each flight of this staff member
            for flight in range(Nvmax+1):
                # If the staff member "m" is in the flight "flight"
                if res_array[m][flight] == i:
                    if is_pilot(m): # If the staff member is a pilot
                        pilots_in_flight.append(m)
                    if not is_pilot(m): # If the staff member is a crew member
                        crew_in_flight.append(m)
                        
        print("Flight number {}: pilots {} cabin crew {} (≥{})\n".format(i, pilots_in_flight, crew_in_flight, Nec[i]))
        
        # Resets the two arrays for the next flight
        pilots_in_flight = []
        crew_in_flight = []
    
# Print the schedule for each crew member
# Print the staff members in each flight and the minimum number of crew members that should be on the flight
print_flights_member(flights_member)
print_pilots_crew(flights_member)


Staff number 0: scheduled at 

Staff number 1: scheduled at 

Staff number 2: scheduled at 



IndexError: index 3 is out of bounds for axis 0 with size 3